In [1]:
from htm.algorithms import TemporalMemory
from htm_rl.modules.htm.temporal_memory import DelayedFeedbackTM
from htm_rl.common.sdr_encoders import IntBucketEncoder
from htm.bindings.sdr import SDR
import numpy as np
import matplotlib.pyplot as plt
from htm_rl.modules.htm.spatial_pooler import UnionTemporalPooler

In [15]:
def make_sdr(pos: tuple, _shape: tuple) -> SDR:
    result = SDR(_shape)
    numpy_res = result.dense
    numpy_res = np.zeros(_shape)
    numpy_res[pos] = 1
    result.dense = numpy_res
    return result

def make_sdrs(array: np.ndarray, _shape: tuple) -> np.ndarray:
    result = np.ndarray((array.size,), dtype=SDR)
    iterator = 0
    for number in array:
        result[iterator] = make_sdr(number, _shape)
        iterator += 1
    return result

def learn_model(tm: TemporalMemory, sdrs: np.ndarray, num_epochs=10) -> list:
    errors = []
    for epoch in range(num_epochs):
        for sdr in sdrs:
            tm.compute(sdr, learn=True)
            tm.activateDendrites(True)
            errors.append(tm.anomaly)
        tm.compute(SDR(sdrs[0].dense.shape), learn=False)
    return errors

def generate_data(n, n_actions, n_states, randomness=1.0, seed=0):
    raw_data = list()
    np.random.seed(seed)
    seed_seq = np.random.randint(0, n_actions, n_states)
    raw_data.append(seed_seq.copy())
    n_replace = int(n_states * randomness)
    for i in range(1, n):
        new_seq = np.random.randint(0, n_actions, n_states)
        if randomness == 1.0:
            raw_data.append(new_seq)
        else:
            indices = np.random.randint(0, n_states, n_replace)
            seed_seq[indices] = new_seq[indices]
            raw_data.append(seed_seq.copy())
    data = [list(zip(range(n_states), x)) for x in raw_data]
    return raw_data, data
def compute(tm, state, action, learn=True):
    context = state_encoder.encode(state)
    active_input = action_encoder.encode(action)

    tm.set_active_context_cells(context)

    tm.activate_basal_dendrites(learn)
    tm.predict_cells()

    tm.set_active_columns(active_input)
    tm.activate_cells(learn)
def run(tm, policy, state_encoder, action_encoder,  learn=True):
    for state, action in policy:
        context = state_encoder.encode(state)
        active_input = action_encoder.encode(action)

        tm.set_active_context_cells(context)

        tm.activate_basal_dendrites(learn)
        tm.predict_cells()

        tm.set_active_columns(active_input)
        tm.activate_cells(learn)




In [4]:
shape = (4,)
cells_per_col = 4

row_data, data = generate_data(4, shape[0], 10)
print(f'row: {row_data[:1]}\n')
print(f'data: {data[:1]}\n')

row: [array([0, 3, 1, 0, 3, 3, 3, 3, 1, 3])]

data: [[(0, 0), (1, 3), (2, 1), (3, 0), (4, 3), (5, 3), (6, 3), (7, 3), (8, 1), (9, 3)]]



In [5]:
n_actions = 4
action_bucket = 10
n_states = 25
state_bucket = 3
action_encoder = IntBucketEncoder(n_actions, action_bucket)
state_encoder = IntBucketEncoder(n_states, state_bucket)

input_columns = action_encoder.output_sdr_size
cells_per_column = 16
output_columns = 4000
output_union_sparsity = 0.01
noise_tolerance_apical = 0.1
learning_margin_apical = 0.2
config_tm = dict(columns=input_columns,
                 cells_per_column=cells_per_column,
                 context_cells=state_encoder.output_sdr_size,
                 feedback_cells=output_columns,
                 activation_threshold_basal=state_bucket,
                 learning_threshold_basal=state_bucket,
                 activation_threshold_apical=int(
                     output_union_sparsity * output_columns * (1 - noise_tolerance_apical)),
                 learning_threshold_apical=int(
                     output_union_sparsity * output_columns * (1 - learning_margin_apical)),
                 connected_threshold_basal=0.5,
                 permanence_increment_basal=0.1,
                 permanence_decrement_basal=0.01,
                 initial_permanence_basal=0.4,
                 predicted_segment_decrement_basal=0.001,
                 sample_size_basal=state_bucket,
                 max_synapses_per_segment_basal=state_bucket,
                 max_segments_per_cell_basal=32,
                 connected_threshold_apical=0.5,
                 permanence_increment_apical=0.1,
                 permanence_decrement_apical=0.01,
                 initial_permanence_apical=0.4,
                 predicted_segment_decrement_apical=0.001,
                 sample_size_apical=int(output_union_sparsity * output_columns),
                 max_synapses_per_segment_apical=int(output_union_sparsity * output_columns),
                 max_segments_per_cell_apical=32,
                 prune_zero_synapses=True,
                 timeseries=False,
                 anomaly_window=1000,
                 confidence_window=1000,
                 noise_tolerance=0.0,
                 sm_ac=0.99)

In [16]:
# tm = TemporalMemory(
#     columnDimensions=shape,
#     cellsPerColumn=cells_per_col,
#     minThreshold=1,
#     activationThreshold=1,
# )
tm = DelayedFeedbackTM(**config_tm)

errors = run(tm, data[0], state_encoder, action_encoder)
print(data)

[[(0, 0), (1, 3), (2, 1), (3, 0), (4, 3), (5, 3), (6, 3), (7, 3), (8, 1), (9, 3)], [(0, 1), (1, 2), (2, 0), (3, 3), (4, 2), (5, 0), (6, 0), (7, 0), (8, 2), (9, 1)], [(0, 2), (1, 3), (2, 3), (3, 2), (4, 0), (5, 1), (6, 1), (7, 1), (8, 1), (9, 0)], [(0, 1), (1, 0), (2, 3), (3, 0), (4, 3), (5, 1), (6, 2), (7, 3), (8, 3), (9, 0)]]


In [7]:
plt.plot(range(len(errors)), errors)
plt.title('tm_train_errors')

NameError: name 'errors' is not defined

In [33]:
tp = UnionTemporalPooler(
    activeOverlapWeight=1.0,
    predictedActiveOverlapWeight=1.0,
    seed=42
)